# Fine-tuning BERT for multi-label text classification English

In [1]:
# !pip install -q transformers datasets

## Load dataset

Next, let's download a multi-label text classification dataset [MultiEURLEX](https://huggingface.co/datasets/multi_eurlex)


In [5]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
from datasets import load_dataset

dataset = load_dataset("multi_eurlex", "en")

Reusing dataset multi_eurlex (/home/davo/.cache/huggingface/datasets/multi_eurlex/en/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [7]:

dataset = dataset.rename_column("labels", "old_labels")

In [8]:

dataset

DatasetDict({
    train: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 55000
    })
    test: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 5000
    })
})

Let's check the first example of the training split:

In [9]:
example = dataset['train'][0]
example

{'celex_id': '32006D0213',
 'text': 'COMMISSION DECISION\nof 6 March 2006\nestablishing the classes of reaction-to-fire performance for certain construction products as regards wood flooring and solid wood panelling and cladding\n(notified under document number C(2006) 655)\n(Text with EEA relevance)\n(2006/213/EC)\nTHE COMMISSION OF THE EUROPEAN COMMUNITIES,\nHaving regard to the Treaty establishing the European Community,\nHaving regard to Directive 89/106/EEC of 21 December 1988, on the approximation of laws, regulations and administrative provisions of the Member States relating to construction products (1), and in particular Article 20(2) thereof,\nWhereas:\n(1)\nDirective 89/106/EEC envisages that in order to take account of different levels of protection for construction works at national, regional or local level, it may be necessary to establish in the interpretative documents classes corresponding to the performance of products in respect of each essential requirement. Those d

## Lables for EUROVOC concepts (21 categories)

In [10]:
labels = ['social questions',
 'industry',
 'finance',
 'trade',
 'business and competition',
 'international relations',
 'agriculture, forestry and fisheries',
 'production, technology and research',
 'transport',
 'employment and working conditions',
 'politics',
 'law',
 'education and communications',
 'international organisations',
 'energy',
 'EUROPEAN UNION',
 'science',
 'agri-foodstuffs',
 'geography',
 'economics',
 'environment']
id2label = {0: 'social questions',
 1: 'industry',
 2: 'finance',
 3: 'trade',
 4: 'business and competition',
 5: 'international relations',
 6: 'agriculture, forestry and fisheries',
 7: 'production, technology and research',
 8: 'transport',
 9: 'employment and working conditions',
 10: 'politics',
 11: 'law',
 12: 'education and communications',
 13: 'international organisations',
 14: 'energy',
 15: 'EUROPEAN UNION',
 16: 'science',
 17: 'agri-foodstuffs',
 18: 'geography',
 19: 'economics',
 20: 'environment'}
label2id = {'social questions': 0,
 'industry': 1,
 'finance': 2,
 'trade': 3,
 'business and competition': 4,
 'international relations': 5,
 'agriculture, forestry and fisheries': 6,
 'production, technology and research': 7,
 'transport': 8,
 'employment and working conditions': 9,
 'politics': 10,
 'law': 11,
 'education and communications': 12,
 'international organisations': 13,
 'energy': 14,
 'EUROPEAN UNION': 15,
 'science': 16,
 'agri-foodstuffs': 17,
 'geography': 18,
 'economics': 19,
 'environment': 20}
labels

['social questions',
 'industry',
 'finance',
 'trade',
 'business and competition',
 'international relations',
 'agriculture, forestry and fisheries',
 'production, technology and research',
 'transport',
 'employment and working conditions',
 'politics',
 'law',
 'education and communications',
 'international organisations',
 'energy',
 'EUROPEAN UNION',
 'science',
 'agri-foodstuffs',
 'geography',
 'economics',
 'environment']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [11]:
def numbers_to_classes(l=[0,10,20]):
  zero_cl = [0.0] * 21
  for i in l:
    zero_cl[i] = 1.0
  
  return np.array(zero_cl, dtype=np.float)

In [12]:
numbers_to_classes(l=[0,10,20])

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1.])

In [13]:
from transformers import AutoTokenizer
import numpy as np
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=300)
  # add labels

  labels_matrix = []
  for r in examples["old_labels"]:
    labels_matrix.append(np.array(numbers_to_classes(r), dtype=float))
  # print(labels_matrix)
  encoding["labels"] = np.array(labels_matrix, dtype=float)
  
  return encoding

In [14]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Parameter 'function'=<function preprocess_data at 0x7f1de523e950> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/55 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [15]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['celex_id', 'text', 'old_labels', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [16]:
tokenizer.decode(example['input_ids'])

'[CLS] commission decision of 6 march 2006 establishing the classes of reaction - to - fire performance for certain construction products as regards wood flooring and solid wood panelling and cladding ( notified under document number c ( 2006 ) 655 ) ( text with eea relevance ) ( 2006 / 213 / ec ) the commission of the european communities, having regard to the treaty establishing the european community, having regard to directive 89 / 106 / eec of 21 december 1988, on the approximation of laws, regulations and administrative provisions of the member states relating to construction products ( 1 ), and in particular article 20 ( 2 ) thereof, whereas : ( 1 ) directive 89 / 106 / eec envisages that in order to take account of different levels of protection for construction works at national, regional or local level, it may be necessary to establish in the interpretative documents classes corresponding to the performance of products in respect of each essential requirement. those documents

In [17]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['social questions',
 'industry',
 'trade',
 'production, technology and research',
 'environment']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [18]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [20]:
batch_size = 16
metric_name = "f1"

In [21]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [22]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [23]:
encoded_dataset['train']['labels'][0]

tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.])

In [24]:

encoded_dataset['train']['labels']

tensor([[1., 1., 0.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [25]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  3222,  3247,  1997,  1020,  2233,  2294,  7411,  1996,  4280,
         1997,  4668,  1011,  2000,  1011,  2543,  2836,  2005,  3056,  2810,
         3688,  2004, 12362,  3536,  2723,  2075,  1998,  5024,  3536,  5997,
         2989,  1998, 13681,  4667,  1006, 19488,  2104,  6254,  2193,  1039,
         1006,  2294,  1007,  3515,  2629,  1007,  1006,  3793,  2007, 25212,
         2050, 21923,  1007,  1006,  2294,  1013, 19883,  1013, 14925,  1007,
         1996,  3222,  1997,  1996,  2647,  4279,  1010,  2383,  7634,  2000,
         1996,  5036,  7411,  1996,  2647,  2451,  1010,  2383,  7634,  2000,
        16449,  6486,  1013, 10114,  1013, 25212,  2278,  1997,  2538,  2285,
         2997,  1010,  2006,  1996, 20167,  1997,  4277,  1010,  7040,  1998,
         3831,  8910,  1997,  1996,  2266,  2163,  8800,  2000,  2810,  3688,
         1006,  1015,  1007,  1010,  1998,  1999,  3327,  3720,  2322,  1006,
         1016,  1007, 21739,  1010,  6168,  1024,  1006,  1015, 

In [26]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train']['labels'][0].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6990, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), logits=tensor([[ 0.7119, -0.4688,  0.4276,  0.1330,  0.2932,  0.0481, -0.4633, -0.3265,
          0.0462, -0.1536, -0.1575,  0.1022, -0.5690, -0.4023,  0.2750,  0.1596,
         -0.2457, -0.2454,  0.6577, -0.1985, -0.0819]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

Let's start training!

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [28]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, celex_id, old_labels. If text, celex_id, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/davo/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 55000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 34380
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as:

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.143100,0.176575,0.805769,0.860744,0.278400
2,0.119000,0.165901,0.822025,0.878607,0.297200
3,0.102900,0.168147,0.822769,0.879608,0.280400
4,0.089300,0.171212,0.826784,0.884181,0.295000
5,0.076500,0.176813,0.827526,0.884803,0.313600
6,0.066200,0.181694,0.826149,0.886337,0.294000
7,0.057200,0.187533,0.824919,0.887558,0.305800
8,0.052200,0.194465,0.824165,0.887693,0.304000
9,0.044600,0.197655,0.823381,0.885266,0.302400
10,0.041600,0.200037,0.824209,0.885911,0.307400


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, celex_id, old_labels. If text, celex_id, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-3438
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-3438/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-3438/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-3438/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-3438/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, celex_id, old_labels. If



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from bert-finetuned-sem_eval-english/checkpoint-17190 (score: 0.8275255890442031).


TrainOutput(global_step=34380, training_loss=0.08310864580586041, metrics={'train_runtime': 13399.127, 'train_samples_per_second': 41.047, 'train_steps_per_second': 2.566, 'total_flos': 8.480611359e+16, 'train_loss': 0.08310864580586041, 'epoch': 10.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [29]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, celex_id, old_labels. If text, celex_id, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


{'eval_loss': 0.17681264877319336,
 'eval_f1': 0.8275255890442031,
 'eval_roc_auc': 0.8848033126767937,
 'eval_accuracy': 0.3136,
 'eval_runtime': 39.4796,
 'eval_samples_per_second': 126.648,
 'eval_steps_per_second': 7.928,
 'epoch': 10.0}

## Inference

Let's test the model on a new sentence:

In [30]:
dataset['test']['text'][0]

'COUNCIL REGULATION (EU) No 1390/2013\nof 16 December 2013\non the allocation of fishing opportunities under the Protocol agreed between the European Union and the Union of the Comoros setting out the fishing opportunities and financial contribution provided for in the Fisheries Partnership Agreement currently in force between the two parties\nTHE COUNCIL OF THE EUROPEAN UNION,\nHaving regard to the Treaty on the Functioning of the European Union, and in particular Article 43(3) thereof,\nHaving regard to the proposal from the European Commission,\nWhereas:\n(1)\nOn 5 October 2006, the Council approved the conclusion of the Partnership Agreement in the fisheries sector between the European Community and the Union of the Comoros (the ‘Partnership Agreement’) by adopting Regulation (EC) No 1563/2006 (1).\n(2)\nThe European Union negotiated with the Union of the Comoros a new Protocol to the Partnership Agreement granting vessels of the European Union fishing opportunities in Comoros wate

### Below you can observe a few examples with their predicted outputs for Bert model and what actually was true label

In [31]:
def predict_labels(text, thres_prob=0.5):
    encoding = tokenizer(text, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = trainer.model(**encoding)
    logits = outputs.logits
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= thres_prob)] = 1
    # turn predicted id's into actual label names
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

    return predicted_labels

In [32]:
predicted_labels = predict_labels(text=dataset['test']['text'][0][:512])

In [33]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['international relations', 'agriculture, forestry and fisheries', 'geography']


In [34]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][0])) if label == 1.0])

true labels>>> ['international relations', 'agriculture, forestry and fisheries', 'EUROPEAN UNION', 'geography']


### In the case above Bert model was able to pick up all true categories but missed 'EUROPEAN UNION'  category

In [35]:
predicted_labels = predict_labels(text=dataset['test']['text'][100][:512])

In [36]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['trade', 'agri-foodstuffs', 'geography']


In [37]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][100])) if label == 1.0])

true labels>>> ['trade', 'agriculture, forestry and fisheries', 'production, technology and research', 'agri-foodstuffs', 'geography']


### In the case above Bert model was able to pick up all true categories but missed 'production, technology and research'  and agriculture, forestry and fisheries categories

In [38]:
predicted_labels = predict_labels(text=dataset['test']['text'][1000][:512])

In [39]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['international relations', 'EUROPEAN UNION', 'geography']


In [40]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][1000])) if label == 1.0])

true labels>>> ['trade', 'international relations', 'EUROPEAN UNION', 'geography']


### In the case above Bert model was able to pick up all true categories but also missed 'trade' category

In [44]:
predicted_labels = predict_labels(text=dataset['test']['text'][1111][:512])

In [45]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['social questions', 'trade', 'agri-foodstuffs']


In [46]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][1111])) if label == 1.0])

true labels>>> ['trade', 'agri-foodstuffs']


## we capture trade and agri-foodstuffs but mislabel social questions category

In [47]:
predicted_labels = predict_labels(text=dataset['test']['text'][2230][:512])

In [48]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['trade', 'agri-foodstuffs']


In [49]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][2230])) if label == 1.0])

true labels>>> ['trade', 'agri-foodstuffs']


### Based on output above we can observe that Bert returns also fully correct list of labels as well.

### Based on examples above it is clear thta BERT model learned to pick up majority of the categories but very often missed one category from the true category list.  Good part Bert picks up majority of the classes, bad it often misses one class

## Metrics that would be useful for multilabel classification are F1 score and AUC/ROC curve. We will stick to F1

In [52]:

{'eval_loss': 0.17681264877319336,
 'eval_f1': 0.8275255890442031,
 'eval_roc_auc': 0.8848033126767937,
 'eval_accuracy': 0.3136,
 'eval_runtime': 39.4796,
 'eval_samples_per_second': 126.648,
 'eval_steps_per_second': 7.928,
 'epoch': 10.0}

{'eval_loss': 0.17681264877319336,
 'eval_f1': 0.8275255890442031,
 'eval_roc_auc': 0.8848033126767937,
 'eval_accuracy': 0.3136,
 'eval_runtime': 39.4796,
 'eval_samples_per_second': 126.648,
 'eval_steps_per_second': 7.928,
 'epoch': 10.0}